### download the rolling hindcasts 

In [24]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [25]:
import sys
import pathlib
from datetime import datetime
from dateutil.relativedelta import relativedelta

In [26]:
import numpy as np

In [27]:
HOME = pathlib.Path.home()

In [28]:
CWD = pathlib.Path.cwd()

In [29]:
CWD

PosixPath('/home/nicolasf/operational/ICU/development/hotspots/code/ICU_Water_Watch/notebooks/C3S')

### import local functions for the downloading of the C3S forecasts 

In [30]:
sys.path.append('../..')

In [31]:
from ICU_Water_Watch import C3S, domains

### show the predefined domains 

In [32]:
domains.domains

{'Tropical_Pacific': [140, 220, -25, 25],
 'SW_Pacific': [172.5, 190, -22.5, -12],
 'Fiji': [175, 183, -21, -15],
 'NZ': [161, 181, -50, -30],
 'Pacific': [140, 240, -50, 25],
 'C3S_download': [100, 240, -50, 30],
 'Water_Watch': [120, 240, -38, 22.5]}

### provider (always CDS for now)

In [33]:
provider = 'CDS'

### variable name

In [34]:
var_name = 'tprate'

### mapping GCMs to systems 

In [38]:
from collections import OrderedDict as OD

In [39]:
dict_systems = OD(ECMWF=5,
                   UKMO=600, 
                   METEO_FRANCE=8, 
                   DWD=21, 
                   CMCC=35, 
                   NCEP=2,
                   JMA=2, 
                   )

In [40]:
dict_systems.keys()

odict_keys(['ECMWF', 'UKMO', 'METEO_FRANCE', 'DWD', 'CMCC', 'NCEP', 'JMA'])

### lag in months (if need to download older forecasts)

In [41]:
lag = 0

### get today's date 

In [42]:
date = datetime.utcnow()

### apply lag 

In [43]:
date = date - relativedelta(months=lag)

In [44]:
date

datetime.datetime(2021, 9, 20, 2, 48, 26, 437175)

### generate the dates for the download of the rolling hindcasts 

In [45]:
dates = [datetime(y, date.month, 1) for y in range(1993, 2016 + 1)]

In [46]:
dates

[datetime.datetime(1993, 9, 1, 0, 0),
 datetime.datetime(1994, 9, 1, 0, 0),
 datetime.datetime(1995, 9, 1, 0, 0),
 datetime.datetime(1996, 9, 1, 0, 0),
 datetime.datetime(1997, 9, 1, 0, 0),
 datetime.datetime(1998, 9, 1, 0, 0),
 datetime.datetime(1999, 9, 1, 0, 0),
 datetime.datetime(2000, 9, 1, 0, 0),
 datetime.datetime(2001, 9, 1, 0, 0),
 datetime.datetime(2002, 9, 1, 0, 0),
 datetime.datetime(2003, 9, 1, 0, 0),
 datetime.datetime(2004, 9, 1, 0, 0),
 datetime.datetime(2005, 9, 1, 0, 0),
 datetime.datetime(2006, 9, 1, 0, 0),
 datetime.datetime(2007, 9, 1, 0, 0),
 datetime.datetime(2008, 9, 1, 0, 0),
 datetime.datetime(2009, 9, 1, 0, 0),
 datetime.datetime(2010, 9, 1, 0, 0),
 datetime.datetime(2011, 9, 1, 0, 0),
 datetime.datetime(2012, 9, 1, 0, 0),
 datetime.datetime(2013, 9, 1, 0, 0),
 datetime.datetime(2014, 9, 1, 0, 0),
 datetime.datetime(2015, 9, 1, 0, 0),
 datetime.datetime(2016, 9, 1, 0, 0)]

### path to download the rolling hindcasts 

In [47]:
gcm_path = pathlib.Path(f'/media/nicolasf/END19101/ICU/data/{provider}/operational/hindcasts')

In [48]:
if not(gcm_path.exists()): 
    gcm_path.mkdir(parents=True)

### Now loops over the list of GCM and downloads, for the parameters provided 

In [ ]:
lfiles = []
for date in dates:
    for GCM in dict_systems.keys(): 
        f = C3S.download(
        GCM=GCM,
        system=dict_systems[GCM],
        varname=var_name,
        year=date.year,
        month=date.month,
        leadtimes=[1, 2, 3, 4, 5, 6],
        opath=gcm_path.joinpath(GCM).joinpath(var_name.upper()),
        domain=domains.domains['C3S_download'],
        file_format='netcdf',
        level='surface',
        max_retry=3,
        )   
        lfiles.append(f)


/media/nicolasf/END19101/ICU/data/CDS/operational/hindcasts/ECMWF/TPRATE/ensemble_seas_forecasts_tprate_from_1993_09_ECMWF.netcdf exists already on disk, skipping download and returning path


/media/nicolasf/END19101/ICU/data/CDS/operational/hindcasts/UKMO/TPRATE/ensemble_seas_forecasts_tprate_from_1993_09_UKMO.netcdf exists already on disk, skipping download and returning path


attempting to download tprate for GCM METEO_FRANCE, for level surface, year 1993, month 9, in netcdf, attempt 1 of 3



2021-09-20 14:48:46,652 INFO Welcome to the CDS
2021-09-20 14:48:46,654 INFO Sending request to https://cds.climate.copernicus.eu/api/v2/resources/seasonal-monthly-single-levels
2021-09-20 14:48:47,163 INFO Request is queued
2021-09-20 14:49:39,287 INFO Request is completed
2021-09-20 14:49:39,290 INFO Downloading https://download-0003.copernicus-climate.eu/cache-compute-0003/cache/data1/adaptor.mars.external-1632106178.0847414-21262-4-9d0df15d-389d-45ae-a006-9cb86b285421.nc to /media/nicolasf/END19101/ICU/data/CDS/operational/hindcasts/METEO_FRANCE/TPRATE/ensemble_seas_forecasts_tprate_from_1993_09_METEO_FRANCE.netcdf (2.7M)
2021-09-20 14:49:48,032 INFO Download rate 319.2K/s 



/media/nicolasf/END19101/ICU/data/CDS/operational/hindcasts/METEO_FRANCE/TPRATE/ensemble_seas_forecasts_tprate_from_1993_09_METEO_FRANCE.netcdf downloaded OK


/media/nicolasf/END19101/ICU/data/CDS/operational/hindcasts/DWD/TPRATE/ensemble_seas_forecasts_tprate_from_1993_09_DWD.netcdf exists already on disk, skipping download and returning path


/media/nicolasf/END19101/ICU/data/CDS/operational/hindcasts/CMCC/TPRATE/ensemble_seas_forecasts_tprate_from_1993_09_CMCC.netcdf exists already on disk, skipping download and returning path


/media/nicolasf/END19101/ICU/data/CDS/operational/hindcasts/NCEP/TPRATE/ensemble_seas_forecasts_tprate_from_1993_09_NCEP.netcdf exists already on disk, skipping download and returning path


/media/nicolasf/END19101/ICU/data/CDS/operational/hindcasts/JMA/TPRATE/ensemble_seas_forecasts_tprate_from_1993_09_JMA.netcdf exists already on disk, skipping download and returning path


/media/nicolasf/END19101/ICU/data/CDS/operational/hindcasts/ECMWF/TPRATE/ensemb

2021-09-20 14:49:48,708 INFO Welcome to the CDS
2021-09-20 14:49:48,710 INFO Sending request to https://cds.climate.copernicus.eu/api/v2/resources/seasonal-monthly-single-levels
2021-09-20 14:49:49,322 INFO Request is queued
2021-09-20 14:50:41,548 INFO Request is completed
2021-09-20 14:50:41,551 INFO Downloading https://download-0009.copernicus-climate.eu/cache-compute-0009/cache/data3/adaptor.mars.external-1632106233.6881137-25923-8-95e051a8-2b6a-4aa0-b83a-9c01c4e2cdc0.nc to /media/nicolasf/END19101/ICU/data/CDS/operational/hindcasts/METEO_FRANCE/TPRATE/ensemble_seas_forecasts_tprate_from_1994_09_METEO_FRANCE.netcdf (2.7M)
2021-09-20 14:50:52,249 INFO Download rate 260.9K/s 



/media/nicolasf/END19101/ICU/data/CDS/operational/hindcasts/METEO_FRANCE/TPRATE/ensemble_seas_forecasts_tprate_from_1994_09_METEO_FRANCE.netcdf downloaded OK


/media/nicolasf/END19101/ICU/data/CDS/operational/hindcasts/DWD/TPRATE/ensemble_seas_forecasts_tprate_from_1994_09_DWD.netcdf exists already on disk, skipping download and returning path


/media/nicolasf/END19101/ICU/data/CDS/operational/hindcasts/CMCC/TPRATE/ensemble_seas_forecasts_tprate_from_1994_09_CMCC.netcdf exists already on disk, skipping download and returning path


/media/nicolasf/END19101/ICU/data/CDS/operational/hindcasts/NCEP/TPRATE/ensemble_seas_forecasts_tprate_from_1994_09_NCEP.netcdf exists already on disk, skipping download and returning path


/media/nicolasf/END19101/ICU/data/CDS/operational/hindcasts/JMA/TPRATE/ensemble_seas_forecasts_tprate_from_1994_09_JMA.netcdf exists already on disk, skipping download and returning path


/media/nicolasf/END19101/ICU/data/CDS/operational/hindcasts/ECMWF/TPRATE/ensemb

...

2021-09-20 15:24:03,154 INFO Welcome to the CDS
2021-09-20 15:24:03,156 INFO Sending request to https://cds.climate.copernicus.eu/api/v2/resources/seasonal-monthly-single-levels
2021-09-20 15:24:03,667 INFO Request is queued
2021-09-20 15:24:09,509 INFO Request is running
2021-09-20 15:24:13,293 INFO Request is completed
2021-09-20 15:24:13,295 INFO Downloading https://download-0001.copernicus-climate.eu/cache-compute-0001/cache/data0/adaptor.mars.external-1632108250.0969677-13847-20-821ba6dc-7c42-4cb4-9c5b-4855e17e7dae.nc to /media/nicolasf/END19101/ICU/data/CDS/operational/hindcasts/JMA/TPRATE/ensemble_seas_forecasts_tprate_from_2008_09_JMA.netcdf (185.3K)
2021-09-20 15:24:14,564 INFO Download rate 146.3K/s



/media/nicolasf/END19101/ICU/data/CDS/operational/hindcasts/JMA/TPRATE/ensemble_seas_forecasts_tprate_from_2008_09_JMA.netcdf downloaded OK


attempting to download tprate for GCM ECMWF, for level surface, year 2009, month 9, in netcdf, attempt 1 of 3



2021-09-20 15:24:15,238 INFO Welcome to the CDS
2021-09-20 15:24:15,240 INFO Sending request to https://cds.climate.copernicus.eu/api/v2/resources/seasonal-monthly-single-levels
2021-09-20 15:24:15,955 INFO Request is queued
2021-09-20 15:24:50,533 INFO Request is completed
2021-09-20 15:24:50,535 INFO Downloading https://download-0013.copernicus-climate.eu/cache-compute-0013/cache/data0/adaptor.mars.external-1632108284.3494043-5018-2-cc070837-6ec8-4c7f-956c-e3885aef5845.nc to /media/nicolasf/END19101/ICU/data/CDS/operational/hindcasts/ECMWF/TPRATE/ensemble_seas_forecasts_tprate_from_2009_09_ECMWF.netcdf (2.7M)
2021-09-20 15:24:59,107 INFO Download rate 325.6K/s 



/media/nicolasf/END19101/ICU/data/CDS/operational/hindcasts/ECMWF/TPRATE/ensemble_seas_forecasts_tprate_from_2009_09_ECMWF.netcdf downloaded OK


attempting to download tprate for GCM UKMO, for level surface, year 2009, month 9, in netcdf, attempt 1 of 3



2021-09-20 15:24:59,782 INFO Welcome to the CDS
2021-09-20 15:24:59,784 INFO Sending request to https://cds.climate.copernicus.eu/api/v2/resources/seasonal-monthly-single-levels
2021-09-20 15:25:00,193 INFO Request is queued
